# Simplified Monte Carlo Draft Simulator

Streamlined Monte Carlo draft simulator for real-time draft decisions.
Uses 80% ESPN + 20% ADP weighting with optimized FLEX calculations.

In [ ]:
# Setup & Configuration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Draft Configuration - UPDATE THESE DURING DRAFT
CONFIG = {
    'n_teams': 14,
    'rounds': 15,
    'my_team_idx': 7,          # 0-based (8th pick = 7)
    'current_global_pick': 0,   # 0-based current pick number
    'my_current_roster': [],    # Add player names as you draft
    'n_sims': 500,             # Simulations per candidate
    'candidate_count': 10       # Top candidates to evaluate
}

# Generate snake draft order
def get_pick_order(n_teams, rounds):
    order = []
    for r in range(rounds):
        if r % 2 == 0:
            order.extend(range(n_teams))
        else:
            order.extend(reversed(range(n_teams)))
    return np.array(order)

PICK_ORDER = get_pick_order(CONFIG['n_teams'], CONFIG['rounds'])
STARTER_REQUIREMENTS = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

print(f"Team #{CONFIG['my_team_idx']+1} | {CONFIG['n_teams']} teams, {CONFIG['rounds']} rounds")
print(f"Lineup: QB(1), RB(2), WR(2), TE(1), FLEX(1), K(1), DST(1)")

In [ ]:
# Load and Merge Player Data
def load_player_data():
    # ESPN rankings
    espn_df = pd.read_csv('../data/espn_projections_20250814.csv')
    espn_df['espn_rank'] = espn_df['overall_rank']
    
    # ADP data
    adp_df = pd.read_csv('../data/fantasypros_adp_20250815.csv')
    adp_df['adp_rank'] = adp_df['RANK']
    adp_df['player_name'] = adp_df['PLAYER']
    
    # Projections
    proj_df = pd.read_csv('../data/projections/projections_all_positions_20250814.csv')
    proj_df['player_name'] = proj_df['PLAYER'].fillna(proj_df['UNNAMED:_0_LEVEL_0_PLAYER'])
    proj_df['player_name'] = proj_df['player_name'].str.replace(r'\s+[A-Z]{2,3}$', '', regex=True).str.strip()
    proj_df['proj'] = proj_df['MISC_FPTS'].fillna(proj_df['FPTS']).fillna(100)
    
    # Merge data
    merged = espn_df[['player_name', 'position', 'espn_rank', 'team']].merge(
        adp_df[['player_name', 'adp_rank']], on='player_name', how='outer'
    ).merge(
        proj_df[['player_name', 'proj', 'POSITION']], on='player_name', how='left'
    )
    
    # Clean up
    merged['pos'] = merged['position'].fillna(merged['POSITION']).fillna('FLEX')
    merged['pos'] = merged['pos'].str.extract(r'([A-Z]+)')[0]
    merged['espn_rank'] = merged['espn_rank'].fillna(300)
    merged['adp_rank'] = merged['adp_rank'].fillna(300)
    merged['proj'] = merged['proj'].fillna(50)
    merged['player_id'] = range(len(merged))
    
    players_df = merged[['player_id', 'player_name', 'pos', 'proj', 'espn_rank', 'adp_rank']].copy()
    players_df.columns = ['player_id', 'name', 'pos', 'proj', 'espn_rank', 'adp_rank']
    players_df = players_df.dropna(subset=['name']).set_index('player_id')
    
    return players_df

# Calculate combined scores (80% ESPN + 20% ADP)
def prepare_player_pool(players_df, top_k=150):
    players_df['espn_score'] = 1.0 / (players_df['espn_rank'] + 1e-6)
    players_df['adp_score'] = 1.0 / (players_df['adp_rank'] + 1e-6)
    players_df['base_score'] = 0.8 * players_df['espn_score'] + 0.2 * players_df['adp_score']
    
    # Select top K by ESPN rank, exclude current roster
    topk_df = players_df.nsmallest(top_k, 'espn_rank')
    available_ids = topk_df.index.to_numpy()
    
    if CONFIG['my_current_roster']:
        roster_mask = topk_df['name'].isin(CONFIG['my_current_roster'])
        available_ids = topk_df[~roster_mask].index.to_numpy()
    
    return available_ids

players_df = load_player_data()
available_ids = prepare_player_pool(players_df)

print(f"Loaded {len(players_df)} players, {len(available_ids)} available")
print(f"Top 5 by projection:")
print(players_df.loc[available_ids].nlargest(5, 'proj')[['name', 'pos', 'proj', 'espn_rank']])

In [ ]:
# Roster Value Calculation (Fixed FLEX)
def compute_team_value(chosen_ids, players_df):
    """Calculate total fantasy points from optimal starting lineup"""
    if len(chosen_ids) == 0:
        return 0.0
    
    df = players_df.loc[chosen_ids]
    bypos = {p: df[df['pos'] == p].sort_values('proj', ascending=False) 
             for p in ['QB', 'RB', 'WR', 'TE', 'K', 'DST']}
    
    total = 0.0
    
    # Starters
    if len(bypos['QB']) > 0: total += float(bypos['QB'].iloc[0]['proj'])
    for i in range(min(2, len(bypos['RB']))): total += float(bypos['RB'].iloc[i]['proj'])
    for i in range(min(2, len(bypos['WR']))): total += float(bypos['WR'].iloc[i]['proj'])
    if len(bypos['TE']) > 0: total += float(bypos['TE'].iloc[0]['proj'])
    if len(bypos['K']) > 0: total += float(bypos['K'].iloc[0]['proj'])
    if len(bypos['DST']) > 0: total += float(bypos['DST'].iloc[0]['proj'])
    
    # FLEX: Best remaining RB/WR/TE
    flex_pool = []
    if len(bypos['RB']) > 2: flex_pool.extend(bypos['RB'].iloc[2:]['proj'])
    if len(bypos['WR']) > 2: flex_pool.extend(bypos['WR'].iloc[2:]['proj'])
    if len(bypos['TE']) > 1: flex_pool.extend(bypos['TE'].iloc[1:]['proj'])
    if flex_pool: total += float(max(flex_pool))
    
    return total

# Test empty roster
print(f"Empty roster value: {compute_team_value([], players_df):.1f}")
print(f"Top 3 players value: {compute_team_value(players_df.nsmallest(3, 'espn_rank').index.tolist(), players_df):.1f}")

In [ ]:
# Monte Carlo Simulation Engine
def get_marginal_value(player_id, current_roster, players_df):
    """Calculate marginal value of adding a player (optimized)"""
    if not current_roster:
        return players_df.loc[player_id]['proj']
    
    current_value = compute_team_value(current_roster, players_df)
    new_value = compute_team_value(current_roster + [player_id], players_df)
    return new_value - current_value

def build_pick_probs(pool, players_df):
    """Build pick probabilities using combined scores"""
    if len(pool) == 0:
        return np.array([])
    scores = players_df.loc[pool]['base_score'].values
    return scores / scores.sum()

def simulate_draft(candidate_id, state_pick_index, players_df, avail_ids, n_sims=500):
    """Simulate remainder of draft with candidate and return expected value"""
    # Get starting roster
    starting_roster_ids = []
    if CONFIG['my_current_roster']:
        for name in CONFIG['my_current_roster']:
            matching = players_df[players_df['name'] == name]
            if len(matching) > 0:
                starting_roster_ids.append(matching.index[0])
    
    # Find my future picks
    my_positions = np.where(PICK_ORDER == CONFIG['my_team_idx'])[0]
    future_picks = my_positions[my_positions > state_pick_index]
    
    evs = []
    availability_count = 0
    
    for sim in range(n_sims):
        np.random.seed(42 + sim)  # Reproducible results
        
        # Initialize simulation
        pool = list(avail_ids.copy())
        my_roster = starting_roster_ids.copy()
        
        # Remove already drafted players
        for p in starting_roster_ids:
            if p in pool: pool.remove(p)
        if candidate_id in pool: pool.remove(candidate_id)
        
        # Draft candidate
        my_roster.append(candidate_id)
        
        # Simulate remaining picks
        for pick_idx in range(state_pick_index + 1, min(len(PICK_ORDER), state_pick_index + 100)):
            if not pool: break
            
            team = PICK_ORDER[pick_idx]
            if team == CONFIG['my_team_idx']:
                # My pick: Choose highest marginal value
                best_player = max(pool, key=lambda p: get_marginal_value(p, my_roster, players_df))
                my_roster.append(best_player)
                pool.remove(best_player)
            else:
                # Other team: Probabilistic pick
                probs = build_pick_probs(pool, players_df)
                if len(probs) > 0:
                    chosen = np.random.choice(pool, p=probs)
                    pool.remove(chosen)
        
        # Calculate final team value
        ev = compute_team_value(my_roster, players_df)
        evs.append(ev)
        
        # Track if candidate would still be available
        if candidate_id in pool:
            availability_count += 1
    
    availability = availability_count / n_sims
    return np.mean(evs), np.std(evs), availability

print("Monte Carlo simulation engine ready!")

In [ ]:
# Evaluate Top Candidates
def evaluate_candidates():
    """Evaluate top candidates and return results"""
    # Select candidates by projection
    candidates = sorted(available_ids, 
                       key=lambda pid: players_df.loc[pid]['proj'], 
                       reverse=True)[:CONFIG['candidate_count']]
    
    results = []
    print(f"Evaluating {len(candidates)} candidates...")
    
    for i, candidate in enumerate(candidates):
        ev, sd, availability = simulate_draft(
            candidate, CONFIG['current_global_pick'], 
            players_df, available_ids, CONFIG['n_sims']
        )
        
        results.append({
            'player_id': candidate,
            'name': players_df.loc[candidate]['name'],
            'pos': players_df.loc[candidate]['pos'],
            'proj': players_df.loc[candidate]['proj'],
            'espn_rank': players_df.loc[candidate]['espn_rank'],
            'ev': ev,
            'sd': sd,
            'availability': availability * 100
        })
        
        if (i + 1) % 3 == 0:
            print(f"  {i+1}/{len(candidates)} complete...")
    
    return pd.DataFrame(results).sort_values('ev', ascending=False)

# Run evaluation
print(f"Starting evaluation at pick #{CONFIG['current_global_pick']+1}")
print(f"Your roster: {CONFIG['my_current_roster'] or 'Empty'}\n")

results_df = evaluate_candidates()
print("\n✅ Evaluation complete!")

In [ ]:
# Visualizations & Decision Summary
def create_visualizations(results_df):
    """Create EV chart and availability heatmap"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Expected Value Chart
    pos_colors = {'QB': 'red', 'RB': 'green', 'WR': 'blue', 'TE': 'orange', 'K': 'purple', 'DST': 'brown'}
    colors = [pos_colors.get(pos, 'gray') for pos in results_df['pos']]
    labels = [f"{row['name']} ({row['pos']})" for _, row in results_df.iterrows()]
    
    bars1 = ax1.barh(labels, results_df['ev'], xerr=results_df['sd'], 
                     color=colors, alpha=0.8, capsize=5)
    ax1.set_xlabel('Expected Team Value (Fantasy Points)')
    ax1.set_title(f'Expected Value by Candidate (Pick #{CONFIG["current_global_pick"]+1})')
    ax1.grid(True, alpha=0.3)
    ax1.invert_yaxis()
    
    # Availability Chart
    avail_colors = ['green' if a > 80 else 'yellow' if a > 50 else 'orange' if a > 20 else 'red' 
                    for a in results_df['availability']]
    
    bars2 = ax2.barh(labels, results_df['availability'], color=avail_colors, alpha=0.8)
    ax2.set_xlabel('Probability Available at Next Pick (%)')
    ax2.set_title('Player Availability Risk')
    ax2.grid(True, alpha=0.3)
    ax2.invert_yaxis()
    
    plt.tight_layout()
    plt.show()

def print_recommendations(results_df):
    """Print decision summary"""
    print("=" * 70)
    print("🏈 DRAFT RECOMMENDATIONS")
    print("=" * 70)
    print(f"Pick #{CONFIG['current_global_pick']+1} | Team #{CONFIG['my_team_idx']+1}")
    print(f"Current Roster: {CONFIG['my_current_roster'] or 'Empty'}")
    
    # Calculate picks until next turn
    my_positions = np.where(PICK_ORDER == CONFIG['my_team_idx'])[0]
    next_picks = my_positions[my_positions > CONFIG['current_global_pick']]
    if len(next_picks) > 0:
        picks_until_next = next_picks[0] - CONFIG['current_global_pick']
        print(f"Picks until your next turn: {picks_until_next}")
    
    print("\n📊 TOP RECOMMENDATIONS:")
    print("-" * 70)
    
    for i, row in results_df.head(5).iterrows():
        # Decision logic
        if row['availability'] > 80:
            decision = "⏸️  WAIT (Very likely available)"
        elif row['availability'] > 50:
            decision = "🤔 CONSIDER (Moderate risk)"
        elif row['availability'] > 20:
            decision = "⚠️  DRAFT NOW (High risk)"
        else:
            decision = "🚨 MUST DRAFT (Won't be available)"
        
        print(f"\n{i+1}. {row['name']} ({row['pos']})")
        print(f"   Projection: {row['proj']:.1f} | ESPN: #{row['espn_rank']:.0f}")
        print(f"   Expected Value: {row['ev']:.1f} ± {row['sd']:.1f}")
        print(f"   Availability: {row['availability']:.0f}%")
        print(f"   {decision}")
    
    print("\n" + "=" * 70)
    print("💡 Update CONFIG['current_global_pick'] and CONFIG['my_current_roster'] after each pick")
    print("=" * 70)

# Display results
create_visualizations(results_df)
print_recommendations(results_df)

In [ ]:
# Quick Update Helper
def update_draft_state(new_pick_number, new_roster_player=None):
    """Helper to quickly update draft state"""
    CONFIG['current_global_pick'] = new_pick_number - 1  # Convert to 0-based
    if new_roster_player:
        CONFIG['my_current_roster'].append(new_roster_player)
    
    print(f"Updated to pick #{new_pick_number}")
    print(f"Current roster: {CONFIG['my_current_roster']}")
    print("Re-run cells 3-6 for updated recommendations")

# Example usage:
# update_draft_state(25, "Ja'Marr Chase")  # After pick 25, you drafted Chase

print("Draft state update helper loaded")
print("Usage: update_draft_state(pick_number, 'Player Name')")